Тема: Mde и sample size

Видео лекции:  
https://www.youtube.com/watch?v=F-6CXV7r1C0

In [1]:
import numpy as np
from tqdm import tqdm_notebook

In [2]:
def check_strat(total_len_sampled, n_iters = 100):
    contain = 0
    
    for _ in tqdm_notebook(range(n_iters)):
    
        general_strata_1_size = 10**7
        general_strata_2_size = 10**6

        len_sampled = int(0.1*(general_strata_1_size + general_strata_2_size))

        distr_1_general = np.random.normal(100,10,general_strata_1_size) #истинное распределение страты 1
        distr_2_general = np.random.normal(50,5,general_strata_2_size)
        
        total_distr_general = np.concatenate((distr_1_general,distr_2_general)) #истинное распределение всех данных
        global_mean = np.mean(total_distr_general)



        indexes = np.random.choice(range(len(total_distr_general)),len_sampled,False)
        distr_1 = total_distr_general[indexes[indexes <= general_strata_1_size]] # распределение сэмлированой 1 страты до 
                                                                                    #аб теста для посчета весов
        distr_2 = total_distr_general[indexes[indexes > general_strata_1_size]] 

        indexes = np.random.choice(range(len(total_distr_general)),len_sampled,False)
        distr_3 = total_distr_general[indexes[indexes <= general_strata_1_size]] # распределение 1 страты во время аб теста из 
                                                                                #которого сэмплируем uhegge
        distr_4 = total_distr_general[indexes[indexes > general_strata_1_size]] 


        total_distr = np.concatenate((distr_1,distr_2)) # распределение до аб теста - для посчета весов
        total_distr_sampled = np.concatenate((distr_3,distr_4)) # распределение во время аб теста из которого сэмплируем

        w1 = len(distr_1)/len(total_distr) # веса необходимо считать до проведения а/б теста иначе возникает аналог  
        w2 = len(distr_2)/len(total_distr) #переобучения - подбор параметра и его использование на одно и той же выборке
        se_acc = [] 
   
        indexes = np.random.choice(range(len(total_distr_sampled)),total_len_sampled,False) # сэмплируе в группу аб
        sample_strat_1 = total_distr_sampled[indexes[indexes <= len(distr_3)]] 
        sample_strat_2 = total_distr_sampled[indexes[indexes > len(distr_3)]]
        var_strat = (np.var(sample_strat_1)*w1 + np.var(sample_strat_2)*w2)/total_len_sampled
        strat_mean = w1 * np.mean(sample_strat_1) + w2 * np.mean(sample_strat_2) #мы получаем расхождение в var/sqrt(N)
                                                                     #от мат ожидания генеральной совокупности, это 
                                                                     # также будет отличаться от Esrs в var/sqrt(N)
        se_strat = np.sqrt(var_strat)
        se_acc.append(se_strat)
        inside = 1
        if global_mean < strat_mean - 2*se_strat or global_mean > strat_mean + 2*se_strat:
            inside = 0
        contain += inside


    print('ratio within interval strat', contain/n_iters)
    print('mean se strat', np.mean(se_acc))

    means = []
    for _ in range(n_iters*10):
        sampled = np.random.choice(total_distr_sampled,total_len_sampled,True)
        means.append(np.mean(sampled))
    print('se bootstrap',np.std(means))
    

In [3]:
check_strat(1000,100)
check_strat(10000,100)
check_strat(100000,100)
check_strat(200000,100)


/tmp/ipykernel_6932/2778226362.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(n_iters)):


  0%|          | 0/100 [00:00<?, ?it/s]

ratio within interval strat 0.96
mean se strat 0.29828574372825395
se bootstrap 0.5555526588200861


  0%|          | 0/100 [00:00<?, ?it/s]

ratio within interval strat 0.98
mean se strat 0.09649027931319162
se bootstrap 0.16577061587107955


  0%|          | 0/100 [00:00<?, ?it/s]

ratio within interval strat 0.98
mean se strat 0.03047059508913767
se bootstrap 0.05455785960571768


  0%|          | 0/100 [00:00<?, ?it/s]

ratio within interval strat 0.84
mean se strat 0.02162026880989881
se bootstrap 0.0393410346205415


In [4]:
check_strat(1000,2)

/tmp/ipykernel_5939/2778226362.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(n_iters)):


  0%|          | 0/2 [00:00<?, ?it/s]

ratio within interval strat 1.0
mean se strat 0.2959438703169033
se bootstrap 0.33920851289701603
